The aim of this experiment is simply to test that MS-RWM outperforms MS-RWM on the G and K example.

In [1]:
import numpy as np
from numpy.random import rand, randn
from numpy import ones, exp, log, diag, vstack, pi, array, r_, isfinite, logspace, zeros, eye, quantile
from numpy.linalg import norm, solve
from numpy.random import default_rng, choice, randint
from scipy.optimize import fsolve
from scipy.stats import multivariate_normal as MVN
from scipy.special import ndtri, ndtr
from scipy.stats import uniform as udist
from scipy.stats import norm as ndist
from scipy.linalg import block_diag

import time
from math import prod
from warnings import catch_warnings, filterwarnings, resetwarnings


import matplotlib.pyplot as plt
from matplotlib import rc
from ipywidgets.widgets import interact


from RWM import RWM
from Manifolds.Manifold import Manifold
from Manifolds.GKManifoldNew import GKManifold, data_generator, find_point_on_manifold, find_point_on_manifold_from_θ
from markov_snippets import MSAdaptiveTolerancesAdaptiveδ, MSAdaptiveTolerancesAdaptiveδSwitchIntegrator
from markov_snippets import SMCAdaptiveTolerancesAdaptiveδ
# from tangential_hug_functions import HugTangentialMultivariate
# from utils import ESS_univariate, prep_contour
from copy import deepcopy

# G and K settings

In [2]:
def generate_setting(m, B, δ, N, ϵ0=20, thinning=10, thug=True, verbose=True, initialization='init_RWMϵ0', burnin=100, δ0=0.01, kernel_type='uniform', ϵmin=1e-4, quantile_value=0.01, maxiter=50, ϵprop_switch=0.05, pmoved_switch=0.15, switch_strategy='ap', δmin=1e-4, δmax=100.0, ap_target=0.3, use_autograd=False, init_manifold_prior=True):
    """Generates an object from which one can grab the settings. This allows one to run multiple scenarios."""
    θ0        = array([3.0, 1.0, 2.0, 0.5])      # True parameter value on U(0, 10) scale.
    d         = 4 + m                            # Dimensionality of ξ=(θ, z)
    ystar     = data_generator(θ0, m, seed=1234) # Observed data
    q         = MVN(zeros(d), eye(d))            # Proposal distribution for THUG
    ξ0        = find_point_on_manifold_from_θ(ystar=ystar, θfixed=ndtri(θ0/10), ϵ=1e-5, maxiter=5000, tol=1e-15, kernel_type=kernel_type)
    resetwarnings()
    manifold  = GKManifold(ystar, kernel_type=kernel_type, use_autograd=use_autograd)
    if δ0 is None:
        δ0 = δ
    return {
        'θ0': θ0,
        'm' : m,
        'd' : d,
        'ystar': ystar,
        'q': q,
        'ξ0': ξ0,
        'B': B,
        'δ': δ,
        'N': N,
        'manifold': manifold,
        'thinning': thinning,
        'thug': thug,
        'verbose': verbose,
        'initialization': initialization,
        'burnin': burnin,
        'δ0': δ0,
        'kernel_type': kernel_type,
        'ϵmin': ϵmin,
        'quantile_value': quantile_value,
        'maxiter': maxiter,
        'ϵprop_switch': ϵprop_switch,
        'pmoved_switch': pmoved_switch,
        'switch_strategy': switch_strategy,
        'δmin': δmin,
        'δmax': δmax,
        'ap_target': ap_target,
        'ϵ0': ϵ0,
        'use_autograd': use_autograd,
        'init_manifold_prior': init_manifold_prior
    }

In [5]:
SETTINGS = generate_setting(
    m=50, 
    B=20, 
    δ=0.01, 
    N=5000, 
    thinning=10, 
    δ0=0.01, 
    kernel_type='uniform', 
    thug=False, 
    verbose=True, 
    initialization='init_prior', 
    burnin=100, 
    maxiter=100, 
    ϵmin=1e-8, 
    quantile_value=0.9,
    δmin=1e-3, 
    δmax=10.0, 
    ap_target=0.2
)

# SMC-RWM

In [ ]:
SMC_RWM = SMCAdaptiveTolerancesAdaptiveδ(SETTINGS)
_ = SMC_RWM.sample()

Kernel: RWM.
Initializing particles from prior.
Initial Epsilon: {:.3f} 7714535797754.47
Iteration:  1
	Particles resampled.
	Epsilon: 655160424.777918
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.81420000
	Step-size adapted to: 0.01359477
Iteration:  2
	Particles resampled.
	Epsilon: 123666528.952347
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.75660000
	Step-size adapted to: 0.01795709
Iteration:  3
	Particles resampled.
	Epsilon: 29984766.918943
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.72640000
	Step-size adapted to: 0.02336374
Iteration:  4
	Particles resampled.
	Epsilon: 8907587.134207
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.69960000
	Step-size adapted to: 0.02999363
Iteration:  5
	Particles resampled.
	Epsilon: 1764714.777835
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.70060000
	Step-size adapted to: 0.03852414
Iteration:  6
	Particles resampled.
	Epsi

	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.04700000
	Step-size adapted to: 0.77153736
Iteration:  50
	Particles resampled.
	Epsilon: 20.772332
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.04180000
	Step-size adapted to: 0.71286003
Iteration:  51
	Particles resampled.
	Epsilon: 20.502106
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.04780000
	Step-size adapted to: 0.66062417
Iteration:  52
	Particles resampled.
	Epsilon: 20.303882
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.03480000
	Step-size adapted to: 0.60824946
Iteration:  53
	Particles resampled.
	Epsilon: 20.141447
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.03680000
	Step-size adapted to: 0.56058736
Iteration:  54
	Particles resampled.
	Epsilon: 20.002283
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.04060000
	Step-size adapted to: 0.51764263
Iteration:  55
	Particles resampled.
	Epsil

# Markov Snippets RWM with adaptive step size

In [4]:
MS_RWM = MSAdaptiveTolerancesAdaptiveδ(SETTINGS)
_ = MS_RWM.sample()

Integrator: RWM.
Initializing particles from prior.
Iteration:  1
	Trajectories constructed.
	Epsilon: 28.721902
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.954
	Step-size adapted to: 0.01457613
Iteration:  2
	Trajectories constructed.
	Epsilon: 19.702821
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.993
	Step-size adapted to: 0.02166689
Iteration:  3
	Trajectories constructed.
	Epsilon: 19.361059
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.918
	Step-size adapted to: 0.03102785
Iteration:  4
	Trajectories constructed.
	Epsilon: 15.689542
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.931
	Step-size adapted to: 0.04471393
Iteration:  5
	Trajectories constructed.
	Epsilon: 13.430275
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.919
	Step-size adapte

	Epsilon: 7.423876
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.164
	Step-size adapted to: 0.06437885
Iteration:  44
	Trajectories constructed.
	Epsilon: 7.363268
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.166
	Step-size adapted to: 0.06329999
Iteration:  45
	Trajectories constructed.
	Epsilon: 7.298763
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.169
	Step-size adapted to: 0.06233263
Iteration:  46
	Trajectories constructed.
	Epsilon: 7.226362
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.195
	Step-size adapted to: 0.06216456
Iteration:  47
	Trajectories constructed.
	Epsilon: 7.140091
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.177
	Step-size adapted to: 0.06145991
Iteration:  48
	Trajectories constructed.
	Epsilon: 7.045904
	Weights compute

	Trajectories constructed.
	Epsilon: 4.803794
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.183
	Step-size adapted to: 0.03998423
Iteration:  87
	Trajectories constructed.
	Epsilon: 4.770540
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.186
	Step-size adapted to: 0.03970135
Iteration:  88
	Trajectories constructed.
	Epsilon: 4.729090
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.198
	Step-size adapted to: 0.03965770
Iteration:  89
	Trajectories constructed.
	Epsilon: 4.693806
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.188
	Step-size adapted to: 0.03941652
Iteration:  90


KeyboardInterrupt: 